In [2]:
import pandahouse as ph

### Второй проект задание 3

В базе данных вашей компании присутствуют следующие таблицы:

#### city:
- city_id – id города, где проходит акция  
- client_city_id – id города, где находится покупатель  
- city – город  
- client_city – город, где находится покупатель  

----------------------------------------------
#### client:  
- client_id – id покупателя  
- client_city_id – id города, где находится покупатель  
- birth_date – дата рождения покупателя  
- registration – дата регистрации покупателя  

----------------------------------------------
#### promotion:  
- promotion_id – id акции  
- category_id – id категории  
- promotion_name – акция  
- category_name – категория  
- partner_id – id партнёра  
- partner_name – партнёр  

----------------------------------------------
#### purchase:  
- purchase_id – id покупки  
- partner_id – id партнёра  
- client_id – id покупателя  
- city_id – id города  
- promotion_id – id акции  
- category_id – id категории  
- purchase_date – дата покупки  
- price – цена за единицу товара  
- quantity – число проданных единиц  
- status – статус покупки  

----------------------------------------------
Вам нужно написать запрос, чтобы получить такую таблицу:

- purchase_date – дата покупки  
- purchase_id – id покупки  
- client_id – id покупателя  
- client_age – возраст покупателя  
- client_registration_age – как долго человек пользуется вашими магазинами  
- client_category – содержит new или old, если это первая или последующая покупка соответственно  
- promotion_name – акция  
- category_name – категория  
- partner_name – партнёр  
- client_city – город, где находится покупатель  
- city – город  
- revenue – сумма выручки  
- quantity – число проданных единиц  

При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020

База данных доступна по имени project_variant_2

In [3]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
'database':'project_variant_2',
'user':'student',
'password':'dpo_python_2020'
}

In [26]:
query = '''
 SELECT
    purchase_date, 
    purchase_id, 
    client_id,
    DATEDIFF(year, birth_date, now( )) AS client_age,
    DATEDIFF(month, registration, purchase_date) AS client_registration_age, 
    client_category,
    promotion_name,
    category_name,
    partner_name,
    client_city,
    city,
    revenue,
    quantity
FROM(
    SELECT 
            purchase.purchase_date AS purchase_date, 
            purchase.purchase_id AS purchase_id, 
            purchase.client_id AS client_id,
            toFloat32(purchase.quantity) AS quantity,
            client_category,
            (toFloat32(purchase.price) * toFloat32(purchase.quantity)) AS revenue,
            {db}.client.birth_date AS birth_date,
            {db}.client.registration AS registration,
            purchase.status AS status,
            {db}.promotion.promotion_name AS promotion_name,
            {db}.promotion.category_name AS category_name,
            {db}.promotion.partner_name AS partner_name,
            {db}.city.client_city AS client_city,
            {db}.city.city AS city
                    FROM (
                    SELECT purchase_date, purchase_id, client_id, quantity, price, status, partner_id, city_id,
                        CASE
                            WHEN purchase_date = min_date THEN 'new'
                            ELSE 'old'
                        END AS client_category
                        FROM(
                            SELECT 
                                purchase_date, purchase_id, client_id, quantity, price, status, partner_id, city_id,
                                MIN(purchase_date) over (partition by client_id) AS min_date
                            FROM {db}.purchase)) AS purchase
        LEFT JOIN {db}.client AS client ON purchase.client_id = client.client_id
        LEFT JOIN {db}.promotion AS promotion ON purchase.partner_id = promotion.partner_id
        LEFT JOIN {db}.city AS city ON purchase.city_id = city.city_id AND client.client_city_id = city.client_city_id)
WHERE
    status = 1 and purchase_date BETWEEN '2020-05-01' and '2020-08-01'
LIMIT 10       
'''
ph.read_clickhouse(query=query, connection=connection_default)

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-07-26,918,1,26,1,old,campaign_3,Овощи,Всем партнёрам партнёр,Вена,Минск,615.0,3.0
1,2020-07-20,3002,1,26,1,new,campaign_4,Прочее,KarpovCourses,Вена,Новгород,660.0,3.0
2,2020-07-24,3626,1,26,1,old,campaign_4,Прочее,KarpovCourses,Вена,Ярославль,2280.0,8.0
3,2020-06-08,6111,2,23,19,old,campaign_3,Овощи,Всем партнёрам партнёр,Щучинщина,Минск,360.0,1.0
4,2020-05-21,6702,2,23,18,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Щучинщина,Минск,465.0,3.0
5,2020-05-29,7349,2,23,18,old,campaign_1,Бакалея,Google,Щучинщина,Новгород,2080.0,8.0
6,2020-05-15,7362,2,23,18,old,campaign_3,Овощи,Всем партнёрам партнёр,Щучинщина,Вена,180.0,9.0
7,2020-07-16,8155,2,23,20,old,campaign_1,Бакалея,Google,Щучинщина,Минск,2655.0,9.0
8,2020-05-24,1351,3,26,7,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Ярославль,Новгород,950.0,2.0
9,2020-05-27,793,4,26,2,old,campaign_1,Бакалея,Google,Казань,Киев,2520.0,8.0
